# Custom Gates
This tutorial shows how to extend quasar with user-specified custom gates with parameters (parameter-free custom gates should be build with the `U1` and `U2` static methods of class `Gate`). We will build a composite 2-qubit gate to cover all rotations in $SO(4)$ according to the description in https://arxiv.org/pdf/1203.0722.pdf (Figure 1).

In [1]:
import numpy as np
import collections
import quasar

In terms of more conventional gates, the $SO(4)$ gate can be written as 6x $R_y$ gates (each with parameter $\theta$) and 2x CNOT gates.

In [2]:
circuit1 = quasar.Circuit(N=2)
circuit1.add_gate(T=0, key=0, gate=quasar.Gate.Ry(theta=0.0))
circuit1.add_gate(T=0, key=1, gate=quasar.Gate.Ry(theta=0.0))
circuit1.add_gate(T=1, key=(0,1), gate=quasar.Gate.CNOT)
circuit1.add_gate(T=2, key=0, gate=quasar.Gate.Ry(theta=0.0))
circuit1.add_gate(T=2, key=1, gate=quasar.Gate.Ry(theta=0.0))
circuit1.add_gate(T=3, key=(0,1), gate=quasar.Gate.CNOT)
circuit1.add_gate(T=4, key=0, gate=quasar.Gate.Ry(theta=0.0))
circuit1.add_gate(T=4, key=1, gate=quasar.Gate.Ry(theta=0.0))
print(circuit1)
print('')
print(circuit1.param_str)

T   : |0 |1|2 |3|4 |
                    
|0> : -Ry-@-Ry-@-Ry-
          |    |    
|1> : -Ry-X-Ry-X-Ry-

T   : |0 |1|2 |3|4 |

T     Qubits     Name       Gate      :                    Value
0     (0,)       theta      Ry        :   0.0000000000000000E+00
0     (1,)       theta      Ry        :   0.0000000000000000E+00
2     (0,)       theta      Ry        :   0.0000000000000000E+00
2     (1,)       theta      Ry        :   0.0000000000000000E+00
4     (0,)       theta      Ry        :   0.0000000000000000E+00
4     (1,)       theta      Ry        :   0.0000000000000000E+00



But, it would get old typing all that out over and over again if we built a larger circuit with many $SO(4)$ gates in it. One solution is to declare a recipe for a new `Gate` that directly implements the action of the whole circuit above. To help with that, we have provided functionality for the user to specify the fine details of a `Gate` through the `__init__` function:

In [3]:
help(quasar.Gate.__init__)

Help on function __init__ in module quasar.quasar:

__init__(self, N, Ufun, params, name, ascii_symbols)
    Initializer. Params are set as object attributes.
    
    Params:
        N (int > 0) - the dimensionality of the quantum gate, e.g, 1 for
            1-body, 2 for 2-body, etc.
        Ufun (function of OrderedDict of str : float -> np.ndarray of shape
            (2**N,)*2) - a function which generates the unitary
            matrix for this gate from the current parameter set.
        params (OrderedDict of str : float) - the dictionary of initial
            gate parameters.
        name (str) - a simple name for the gate, e.g., 'CNOT'
        ascii_symbols (list of str of len N) - a list of ASCII symbols for
            each active qubit of the gate, for use in generating textual diagrams, e.g.,
            ['@', 'X'] for CNOT.



The main deliverable is to write a function that takes an `OrderedDict` of params and returns the $2^N \times 2^N$ `np.ndarray` "$\hat U$" of `dtype=np.complex128` (the unitary matrix defining the gate operation) for a given set of parameters. For instance:

In [18]:
def composite_so4_u(params):
    
    theta1 = params['theta1']
    theta2 = params['theta2']
    theta3 = params['theta3']
    theta4 = params['theta4']
    theta5 = params['theta5']
    theta6 = params['theta6']
        
    U12 = np.kron(quasar.Matrix.Ry(theta=theta1), quasar.Matrix.Ry(theta=theta2))
    U34 = np.kron(quasar.Matrix.Ry(theta=theta3), quasar.Matrix.Ry(theta=theta4))
    U56 = np.kron(quasar.Matrix.Ry(theta=theta5), quasar.Matrix.Ry(theta=theta6))
        
    U = np.dot(quasar.Matrix.CNOT, U12)
    U = np.dot(U34, U)
    U = np.dot(quasar.Matrix.CNOT, U)
    U = np.dot(U56, U)
        
    return U

We then write a method to build a custom $SO(4)$ gate, which calls the `Gate` `__init__` method with the $\hat U$ function of the previous block, initial parameters, and a few other attributes declaring size `N`, gate name `name`, and a list of ASCII symbols `ascii_symbols` to use in displaying ASCII circuit diagrams:

In [ ]:
def composite_so4(
    theta1=0.0,
    theta2=0.0,
    theta3=0.0,
    theta4=0.0,
    theta5=0.0,
    theta6=0.0,
    ):
    
    params = collections.OrderedDict([
        ('theta1', theta1),
        ('theta2', theta2),
        ('theta3', theta3),
        ('theta4', theta4),
        ('theta5', theta5),
        ('theta6', theta6),
    ])
    
    return quasar.Gate(
        N=2,
        Ufun=composite_so4_u,
        params=params,
        name='SO4',
        ascii_symbols=['SO4A', 'SO4B']
        ) 

We can now build a much simpler circuit with just 1x composite $SO(4)$ gate:

In [19]:
circuit2 = quasar.Circuit(N=2)
circuit2.add_gate(T=0, key=(0,1), gate=composite_so4())
print(circuit2)
print('')
print(circuit2.param_str)

T   : |0   |
            
|0> : -SO4A-
       |    
|1> : -SO4B-

T   : |0   |

T     Qubits     Name       Gate      :                    Value
0     (0, 1)     theta1     SO4       :   0.0000000000000000E+00
0     (0, 1)     theta2     SO4       :   0.0000000000000000E+00
0     (0, 1)     theta3     SO4       :   0.0000000000000000E+00
0     (0, 1)     theta4     SO4       :   0.0000000000000000E+00
0     (0, 1)     theta5     SO4       :   0.0000000000000000E+00
0     (0, 1)     theta6     SO4       :   0.0000000000000000E+00



You can see the impact of the `ascii_symbols` flag in the output of the `print(circuit2)` statement. Note that the order of parameters of `circuit1` and `circuit2` are logically equivalent. This means we can generate an iterable list of test parameters:

In [25]:
theta = 2.0 * np.pi * np.random.rand(6)
print(theta)

[1.26894415 5.23400846 4.92822378 3.6233016  2.72294261 3.72618642]


And then call `set_param_values` with these parameters for both `circuit1` and `circuit2`. Simulation then indicates that the circuits are functionally equivalent:

In [26]:
circuit1.set_param_values(theta)
circuit2.set_param_values(theta)
wfn1 = circuit1.simulate()
wfn2 = circuit2.simulate()
print('wfn1: %s' % wfn1)
print('wfn2: %s' % wfn2)
print('Fidelity: %24.16E' % np.abs(np.dot(wfn1.conj(), wfn2)**2))

wfn1: [ 0.514168  +0.j  0.55594969+0.j -0.65002155+0.j  0.06342861+0.j]
wfn2: [ 0.514168  +0.j  0.55594969+0.j -0.65002155+0.j  0.06342861+0.j]
Fidelity:   1.0000000000000009E+00


Using the new composite gate can make for shorter codes, can make it easier to set certain parameters, and may improve simulation runtimes (as less gate operations are performed). 